In [5]:
import os 
import tensorflow as tf 
import pandas as pd
import numpy as np
from PIL import Image
from PIL import Image, ImageFile
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Conv2D, MaxPooling2D, Flatten, concatenate

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
def load_image_data(image_filenames, image_size=(128, 128)):
    # Create an empty list to store the image data
    image_data = []
    
    # Iterate over each image
    for image_filename in image_filenames:
        # Open the image using Pillow
        with Image.open(image_filename) as img:
            # Resize the image
            img = img.resize(image_size)
            # Convert the image data to a NumPy array and normalize to range [0, 1]
            img_array = np.array(img) / 255.0
            # Add the image data to the list
            image_data.append(img_array)
    
    # Convert the list of image data to a NumPy array
    image_data = np.array(image_data)
    
    return image_data

In [3]:
data = pd.read_csv('C:/Users/oceanlightai/Desktop/datasets/pet_skin/train/train_skin.csv')

# 텍스트 토큰화 및 패딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data[['breed', 'gender', 'region', 'lesions']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1))
sequences = tokenizer.texts_to_sequences(data[['breed', 'gender', 'region', 'lesions']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1))
max_len = max(len(s) for s in sequences)
sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

In [ ]:
load_image_data(data['Raw data ID'].apply(lambda x: 'C:/Users/oceanlightai/Desktop/datasets/pet_skin/train/train_image/' + x))

In [ ]:
# CNN Model for image input
image_input = Input(shape=(128, 128, 3))  # assuming 64x64 RGB images
cnn = Conv2D(64, (3, 3), activation='relu')(image_input)
cnn = MaxPooling2D(pool_size=(2, 2))(cnn)
cnn = Conv2D(128, (3, 3), activation='relu')(cnn)
cnn = MaxPooling2D(pool_size=(2, 2))(cnn)
cnn = Flatten()(cnn)
cnn = Dense(128, activation='relu')(cnn)

In [ ]:
# RNN Model for text input
text_input = Input(shape=(max_len,))
embedding = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_len)(text_input)
lstm = LSTM(128)(embedding)

In [ ]:
concatenated = concatenate([cnn, lstm])

In [ ]:
output = Dense(10, activation='softmax')(concatenated)

In [ ]:
model = Model(inputs=[image_input, text_input], outputs=output)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_sequences, test_sequences, train_images, test_images = train_test_split(sequences, image_data, test_size=0.2)

In [ ]:
model.fit([train_images, train_sequences], epochs=10, validation_data=([test_images, test_sequences]))